In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time

import moyra as ma
# import moyra.elements as ele
# import moyra.forces as ef
# import moyra.frames as fr
import numpy as np

import sys, os
me.mechanics_printing()

In [2]:
DoFs = 2

# heave and fold angle as degrres of freedom
p = ma.DynamicModelParameters(DoFs)

## pendulum mass
p.m_1 = ma.ModelSymbol(value = 0.3,string = 'm_1')
p.m_2 = ma.ModelSymbol(value = 0.3,string = 'm_2')

## pendulum moment of inertia
p.I_1 = ma.ModelSymbol(value = 0.3,string = 'I_1')
p.I_2 = ma.ModelSymbol(value = 0.3,string = 'I_2')

# pendulum length
p.L_1 = ma.ModelSymbol(value = 0.3,string = 'L_1')
p.L_2 = ma.ModelSymbol(value = 0.3,string = 'L_2')

#gravity vector
p.g  = ma.ModelSymbol(value = 9.81,string = 'g')                     # gravity
p.g_v = ma.ModelMatrix(value =[0,0,-1],length=3,string='g_v') # The stationary point

In [3]:
#define refernce frames
body_1 = ma.frames.HomogenousFrame().R_x(p.q[0]).Translate(0,0,-p.L_1/2)
body_2 = body_1.Translate(0,0,-p.L_1/2).R_x(p.q[1]).Translate(0,0,-p.L_2/2)

In [4]:
#Create Elemnts
link_1 = ma.elements.RigidElement(p.q,body_1,ma.elements.MassMatrix(p.m_1,I_xx=p.I_1),grav_vec=p.g_v*p.g,name='inner link')
link_2 = ma.elements.RigidElement(p.q,body_2,ma.elements.MassMatrix(p.m_2,I_xx=p.I_2),grav_vec=p.g_v*p.g, name ='outer_link')

In [5]:
# %%timeit -n 3 -r 1
# Create the SYmbolic Model
eles = [link_1,link_2]
sm = ma.SymbolicModel.FromElementsAndForces(p.q,eles).subs({p.g_v[0]:0,p.g_v[1]:0,p.g_v[2]:-1})

Generating EoM for Element inner link:RigidElement
Generating EoM for Element outer_link:RigidElement


In [6]:
base_dir = "C:\\Git\\NastranMBD\\"
class_name = "pendulum_reduced"
base_class = "BaseMBD"
sm.to_matlab_class(p,base_dir,class_name,base_class)